In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time

In [2]:
driver=webdriver.Chrome('/Users/oscarwan/Downloads/chromedriver')
#driver = webdriver.Safari()
driver.implicitly_wait(10)

/var/folders/bt/709h7c513ld30l7ynqg_cv0c0000gn/T/ipykernel_86468/3742758978.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('/Users/oscarwan/Downloads/chromedriver')


In [3]:
url = 'https://www.ratemyprofessors.com/search/teachers?query=*&sid=675'
driver.get(url)
# handling cookies
try:
    driver.find_element(By.XPATH, '/html/body/div[5]/div/div/button').click()
except:
    print('No Cookie Popup')


# closing any other popups
try:
    driver.find_element(By.XPATH, '//*[@id="bx-close-inside-1177612"]').click()
except Exception as e:
    print(e)
    print("no popups")

In [5]:
# prof link path
linkPath = '//*[@id="root"]/div/div/div[4]/div[1]/div[1]/div[3]/a[1]'
driver.find_elements(By.CSS_SELECTOR, 'div.SearchResultsPage__StyledSearchResultsPage-vhbycj-0.bgplVn > div.SearchResultsPage__SearchResultsWrapper-vhbycj-1.gxbBpy')[0].find_elements(By.TAG_NAME, 'a')[0].get_attribute('href')

'https://www.ratemyprofessors.com/professor?tid=2171'

In [13]:
#driver.find_element(By.CLASS_NAME, 'Buttons__Button-sc-19xdot-1 PaginationButton__StyledPaginationButton-txi1dr-1 gjQZal').click()
def waitForLoad(inputXPath):
    Wait = WebDriverWait(driver, 20)
    Wait.until(EC.presence_of_element_located((By.XPATH, inputXPath)))
    
bottonPath = '//*[@id="root"]/div/div/div[4]/div[1]/div[1]/div[4]/button'
loadMoreBotton = driver.find_element(By.XPATH, bottonPath)
while True:
    try:
        
        waitForLoad(bottonPath)
        WebDriverWait(driver, 20)
        actions = ActionChains(driver)
        actions.move_to_element(loadMoreBotton).perform()
        
        
        loadMoreBotton.click()
        #driver.find_element(By.CSS_SELECTOR, '#root > div > div > div:nth-child(4) > div.SearchResultsPage__StyledSearchResultsPage-vhbycj-0.bgplVn > div.SearchResultsPage__SearchResultsWrapper-vhbycj-1.gxbBpy > div.SearchResultsPage__AddPromptWrapper-vhbycj-2.dubepU > button').click()
        #time.sleep(2)
    except Exception as e:
        print(e)
        print("End of the page")
        break
    

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000100ef2f44 chromedriver + 3813188
1   chromedriver                        0x0000000100e84ae0 chromedriver + 3361504
2   chromedriver                        0x0000000100b88c5c chromedriver + 232540
3   chromedriver                        0x0000000100b8b934 chromedriver + 244020
4   chromedriver                        0x0000000100b8b79c chromedriver + 243612
5   chromedriver                        0x0000000100b8b9d0 chromedriver + 244176
6   chromedriver                        0x0000000100bbbbbc chromedriver + 441276
7   chromedriver                        0x0000000100bbb01c chromedriver + 438300
8   chromedriver                        0x0000000100bb0f58 chromedriver + 397144
9   chromedriver                        0x0000000100bb0898 chromedriver + 395416
10  chromedriver                        0x0000000100be7f4

In [14]:
df = pd.DataFrame(columns = ['id', 'name','quality','numOfRatings','department','wouldTakeAgainPercent','difficulty'])
for ii, prof in enumerate(driver.find_elements(By.CSS_SELECTOR, 'div.SearchResultsPage__StyledSearchResultsPage-vhbycj-0.bgplVn > div.SearchResultsPage__SearchResultsWrapper-vhbycj-1.gxbBpy')[0].find_elements(By.TAG_NAME, 'a')[:-1]):
    profId = int(prof.get_attribute('href').split('=')[1])
    prof = prof.text.split('\n')
    df.loc[ii] = [profId, prof[3], prof[1], prof[2].split(' ')[0], prof[4], prof[6].strip('%'),prof[8]]

In [1]:
for ii, prof in enumerate(driver.find_elements(By.CSS_SELECTOR, 'div.SearchResultsPage__StyledSearchResultsPage-vhbycj-0.bgplVn > div.SearchResultsPage__SearchResultsWrapper-vhbycj-1.gxbBpy')[0].find_elements(By.TAG_NAME, 'a')[:-1]):
    a = prof.get_attribute('href')

NameError: name 'driver' is not defined

In [15]:
df

,id,name,quality,numOfRatings,department,wouldTakeAgainPercent,difficulty
0,2171,Julia Keefer,4.5,6,Film,N/A,2.8
1,18708,Alan Corns,2.8,6,Economics,N/A,4.5
2,18711,Hasia Diner,3.8,9,Social Science,0,3.2
3,37951,Steve Hutkins,4.5,7,Interdisciplinary Studies,100,2
4,41264,Tyler Volk,3.5,43,Science,50,3
...,...,...,...,...,...,...,...
3392,1388150,Stefanos Geroulanos,3.9,30,History,65,3.9
3393,2649675,George Benaroya,5.0,47,Marketing,98,2.8
3394,2118628,Thomas Reynolds,1.3,6,Liberal Studies,0,4.5
3395,2595628,Joseph Califf,3.8,8,Writing,63,4


In [16]:
df.to_csv('generalProfRatings.csv')